In [ ]:
import librosa
import numpy as np
import os
import pandas as pd
from tensorflow.keras.models import load_model

# 🔹 소음 유형별 데시벨 기준 설정
SPL_REFERENCE = 20e-6  # 0dB 기준 음압
DB_REFERENCE = {
    "차량 경적": 100,      # 기준 100dB @ 1m
    "이륜차 경적": 100,    # 기준 100dB @ 1m
    "사이렌": 100,         # 기준 100dB @ 1m
    "차량 주행음": 90,     # 기준 90dB @ 1m
    "이륜차 주행음": 90,   # 기준 90dB @ 1m
    "기타소음": 85,       # 기타소음 기준 (예시로 낮은 기준 설정)
}

# 🔹 거리 & 방향 판정 함수
def analyze_audio(file_path, model):
    """
    WAV 파일에서 소음 유형에 따라 Peak SPL 또는 RMS SPL을 사용하여 거리 판정을 수행하며, 
    스테레오인 경우 방향 판정도 함께 수행
    """
    try:
        # 🔹 1. 오디오 파일 로드 및 특징 추출
        y, sr = librosa.load(file_path, sr=None, mono=False)  # 오디오 로드
        # 특징 추출 (예: MFCC)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfcc, axis=1)
        
        # 🔹 2. 모델 예측 (소음 유형 분류)
        prediction = model.predict(np.expand_dims(mfcc_mean, axis=0))
        predicted_class = np.argmax(prediction)  # 가장 높은 확률의 클래스를 예측
        noise_type = list(DB_REFERENCE.keys())[predicted_class]  # 예측된 소음 유형

        # 🔹 3. RMS SPL 계산
        is_stereo = len(y.shape) == 2 and y.shape[0] == 2
        if is_stereo:
            left_channel = y[0]
            right_channel = y[1]
            rms_total = np.sqrt(np.mean((left_channel + right_channel) ** 2)) / 2
        else:
            rms_total = np.sqrt(np.mean(y ** 2))
        
        rms_spl = 20 * np.log10(rms_total / SPL_REFERENCE + 1e-6)  # RMS SPL 변환
        peak_amplitude = np.max(np.abs(y))  # 최고 진폭 값
        peak_spl = 20 * np.log10(peak_amplitude / SPL_REFERENCE + 1e-6)  # Peak SPL 변환

        # 🔹 4. 소음 유형에 따라 SPL 사용
        if noise_type in ["차량 경적", "이륜차 경적", "사이렌"]:
            spl_used = peak_spl
        else:
            spl_used = rms_spl
        
        # 🔹 5. 거리 계산
        db_ref = DB_REFERENCE.get(noise_type, 100)  # 기본 100dB 기준 적용
        estimated_distance = 1 * (10 ** ((db_ref - spl_used) / 20))
        estimated_distance = round(estimated_distance, 2)

        # 🔹 6. 거리 알람 설정
        distance_alert = "알람 없음"
        if noise_type in ["차량 경적", "이륜차 경적"] and estimated_distance <= 10:
            distance_alert = "🚨 10미터 알람 활성화"
        elif noise_type == "사이렌" and estimated_distance <= 20:
            distance_alert = "🚨 20미터 알람 활성화"
        elif noise_type in ["차량 주행음", "이륜차 주행음"] and estimated_distance <= 3:
            distance_alert = "🚨 3미터 이내 감지, 알람 활성화"

        # 🔹 7. 방향 판정 (스테레오일 경우)
        direction = None
        if is_stereo:
            rms_left = np.sqrt(np.mean(left_channel ** 2))
            rms_right = np.sqrt(np.mean(right_channel ** 2))
            spl_left = 20 * np.log10(rms_left / SPL_REFERENCE + 1e-6)
            spl_right = 20 * np.log10(rms_right / SPL_REFERENCE + 1e-6)
            db_difference = spl_left - spl_right
            if abs(db_difference) < 1.5:
                direction = "중앙"
            elif 1.5 <= abs(db_difference) < 3:
                direction = "약간 왼쪽" if db_difference > 0 else "약간 오른쪽"
            else:
                direction = "왼쪽" if db_difference > 0 else "오른쪽"
        
        return file_path, noise_type, spl_used, estimated_distance, distance_alert, spl_left, spl_right, db_difference, direction

    except Exception as e:
        print(f"❌ 오류 발생: {file_path} - {e}")
        return file_path, None, None, None, None, None, None, None, None

# 🔹 모델 로드 (사전 훈련된 모델)
model = load_model('your_model.h5')  # 모델 경로

# 🔹 분석할 데이터가 있는 폴더 (Ubuntu 환경)
folder_path = "/path/to/your/data"

# 🔹 분석 결과 CSV 저장 경로
csv_output_path = "analysis_results.csv"

# 🔹 폴더 내 모든 WAV 파일 가져오기 
wav_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".wav")])

# 분석 결과 저장 리스트
results = []

# 모든 WAV 파일 반복 실행
for file_name in wav_files:
    file_path = os.path.join(folder_path, file_name)
    
    # 모델 예측 및 분석
    result = analyze_audio(file_path, model)
    
    # 결과 저장
    results.append(result)

# CSV 저장
df = pd.DataFrame(results, columns=["파일명", "소음 유형", "SPL (dB)", "추정 거리 (m)", "거리 알람", "왼쪽 채널 (dB)", "오른쪽 채널 (dB)", "좌우 차이 (dB)", "판정 방향"])
df.to_csv(csv_output_path, index=False, encoding="utf-8-sig")

print("✅ 분석 완료! 결과가 CSV 파일로 저장되었습니다.")